In [16]:
import pandas as pd

# Load and scale data for different bins
def load_and_scale(file_path, scale_factor):
    df = pd.read_csv(file_path)
    df['HP_kWh'] *= scale_factor
    df['timestamp'] = pd.to_datetime(df['timestamp'])  # Ensure timestamp is in datetime format
    return df

# Load data
try:
    dffnsb = load_and_scale('FNSB_B1.csv', 24293)
    dfkpb = load_and_scale('KPB_B1.csv', 9730)
    dfmatsu = load_and_scale('Mat_Su_B1.csv', 31715)
    dfanc_b1 = load_and_scale('ANC_B1.csv', 14654)
    dfden_b1 = load_and_scale('Denali_B1.csv', 2403)

    dfsal1 = load_and_scale('Salcha_bin1.csv', 275)
    dfsal2 = load_and_scale('Salcha_bin2.csv', 183)
    dfsal3 = load_and_scale('Salcha_bin3.csv', 18)
    dffbk1 = load_and_scale('FBK_bin1.csv', 15840)
    dffbk2 = load_and_scale('FBK_bin2.csv', 853)
    dffbk3 = load_and_scale('FBK_bin3.csv', 10)
    dfnp1 = load_and_scale('NP_bin1.csv', 6329)
    dfnp2 = load_and_scale('NP_bin2.csv', 778)
    dfnp3 = load_and_scale('NP_bin3.csv', 7)
    dfken1 = load_and_scale('Kenai_bin1.csv', 3458)
    dfken2 = load_and_scale('Kenai_bin2.csv', 603)
    dfken3 = load_and_scale('Kenai_bin3.csv', 18)
    dfwas1 = load_and_scale('Wasilla_bin1.csv', 21322)
    dfwas2 = load_and_scale('Wasilla_bin2.csv', 16)
    dfwas3 = load_and_scale('Wasilla_bin3.csv', 2)
    dfnik1 = load_and_scale('Nikiski_bin1.csv', 386)
    dfnik2 = load_and_scale('Nikiski_bin2.csv', 156)
    dfnik3 = load_and_scale('Nikiski_bin3.csv', 12)
    dfsold1 = load_and_scale('Soldotna_bin1.csv', 4653)
    dfsold2 = load_and_scale('Soldotna_bin2.csv', 437)
    dfsold3 = load_and_scale('Soldotna_bin3.csv', 9)
    dfpalm1 = load_and_scale('Palmer_bin1.csv', 9656)
    dfpalm2 = load_and_scale('Palmer_bin2.csv', 115)
    dfpalm3 = load_and_scale('Palmer_bin3.csv', 7)
    dfhoust1 = load_and_scale('Houston_bin1.csv', 503)
    dfhoust2 = load_and_scale('Houston_bin2.csv', 87)
    dfhoust3 = load_and_scale('Houston_bin3.csv', 7)
    dfanc_bin1 = load_and_scale('ANC_bin1.csv', 14134)
    dfanc_bin2 = load_and_scale('ANC_bin2.csv', 472)
    dfanc_bin3 = load_and_scale('ANC_bin3.csv', 48)
    dfden_bin1 = load_and_scale('Denali_bin1.csv', 2401)
    dfden_bin2 = load_and_scale('Denali_bin2.csv', 2)
except KeyError as e:
    print(e)
    raise

# Function to merge dataframes without averaging
def merge_dataframes(dfs):
    df_merged = pd.merge(dfs[0][['timestamp', 'HP_kWh']], dfs[1][['timestamp', 'HP_kWh']], on='timestamp', suffixes=('_1', '_2'))
    for i, df in enumerate(dfs[2:], start=3):  # start=3 because we already have _1 and _2
        df_merged = pd.merge(df_merged, df[['timestamp', 'HP_kWh']], on='timestamp')
        df_merged.rename(columns={'HP_kWh': f'HP_kWh_{i}'}, inplace=True)  # Correctly rename the last merged column
    return df_merged

# Merge for each set
dfsal = merge_dataframes([dfsal1, dfsal2, dfsal3])
dffbk = merge_dataframes([dffbk1, dffbk2, dffbk3])
dfnp = merge_dataframes([dfnp1, dfnp2, dfnp3])
dfken = merge_dataframes([dfken1, dfken2, dfken3])
dfwas = merge_dataframes([dfwas1, dfwas2, dfwas3])
dfpalm = merge_dataframes([dfpalm1, dfpalm2, dfpalm3])
dfsold = merge_dataframes([dfsold1, dfsold2, dfsold3])
dfhoust = merge_dataframes([dfhoust1, dfhoust2, dfhoust3])
dfnik = merge_dataframes([dfnik1, dfnik2, dfnik3])
dfanc = merge_dataframes([dfanc_bin1, dfanc_bin2, dfanc_bin3])
dfden = merge_dataframes([dfden_bin1, dfden_bin2])

# Combining dfb1
dfb1 = pd.DataFrame()
dfb1['timestamp'] = dffnsb['timestamp']
dfb1['Total_HP_kWh'] = (
    dffnsb['HP_kWh'] +
    dfkpb['HP_kWh'] +
    dfmatsu['HP_kWh'] +
    dfanc_b1['HP_kWh'] +
    dfden_b1['HP_kWh']
)

# Combining averages for dfc3
dfc3 = pd.DataFrame()
dfc3['timestamp'] = dfsal['timestamp']
dfc3['Total_HP_kWh'] = (
    dfsal.filter(like='HP_kWh').sum(axis=1) +
    dffbk.filter(like='HP_kWh').sum(axis=1) +
    dfnp.filter(like='HP_kWh').sum(axis=1) +
    dfken.filter(like='HP_kWh').sum(axis=1) +
    dfwas.filter(like='HP_kWh').sum(axis=1) +
    dfpalm.filter(like='HP_kWh').sum(axis=1) +
    dfnik.filter(like='HP_kWh').sum(axis=1) +
    dfhoust.filter(like='HP_kWh').sum(axis=1) +
    dfsold.filter(like='HP_kWh').sum(axis=1) +
    dfanc.filter(like='HP_kWh').sum(axis=1) + 
    dfden.filter(like='HP_kWh').sum(axis=1)
)

# Merging and calculating variances
merged_df = pd.merge(dfb1, dfc3, on='timestamp', suffixes=('_b1', '_c3'))
merged_df['variance'] = (merged_df['Total_HP_kWh_b1'] - merged_df['Total_HP_kWh_c3']).abs()
merged_df['percent_difference'] = merged_df['variance'] / merged_df[['Total_HP_kWh_b1', 'Total_HP_kWh_c3']].mean(axis=1) * 100

# Sorting and selecting top percent differences
top_percent_differences = merged_df.sort_values(by='percent_difference', ascending=False).head(5)

# Display the results
print(top_percent_differences[['timestamp', 'Total_HP_kWh_b1', 'Total_HP_kWh_c3', 'variance', 'percent_difference']])


               timestamp  Total_HP_kWh_b1  Total_HP_kWh_c3     variance  \
5101 2018-08-01 13:30:00      7912.623331      1311.913887  6600.709444   
5076 2018-07-31 12:30:00      7912.623331      1311.913887  6600.709444   
5104 2018-08-01 16:30:00      7912.623331      1311.913887  6600.709444   
5079 2018-07-31 15:30:00      7912.623331      1311.913887  6600.709444   
5077 2018-07-31 13:30:00      7912.623331      1311.913887  6600.709444   

      percent_difference  
5101          143.111991  
5076          143.111991  
5104          143.111991  
5079          143.111991  
5077          143.111991  


In [17]:
import pandas as pd

# Load and scale data for different bins
def load_and_scale(file_path, scale_factor):
    df = pd.read_csv(file_path)
    df['HP_kWh'] *= scale_factor
    df['timestamp'] = pd.to_datetime(df['timestamp'])  # Ensure timestamp is in datetime format
    return df

# Load data
dffnsb = load_and_scale('FNSB_B1.csv', 24293)
dfsal1 = load_and_scale('Salcha_bin1.csv', 275)
dfsal2 = load_and_scale('Salcha_bin2.csv', 183)
dfsal3 = load_and_scale('Salcha_bin3.csv', 18)
dffbk1 = load_and_scale('FBK_bin1.csv', 15840)
dffbk2 = load_and_scale('FBK_bin2.csv', 853)
dffbk3 = load_and_scale('FBK_bin3.csv', 10)
dfnp1 = load_and_scale('NP_bin1.csv', 6329)
dfnp2 = load_and_scale('NP_bin2.csv', 778)
dfnp3 = load_and_scale('NP_bin3.csv', 7)
dfken1 = load_and_scale('Kenai_bin1.csv', 3458)
dfken2 = load_and_scale('Kenai_bin2.csv', 603)
dfken3 = load_and_scale('Kenai_bin3.csv', 18)
dfwas1 = load_and_scale('Wasilla_bin1.csv', 21322)
dfwas2 = load_and_scale('Wasilla_bin2.csv', 16)
dfwas3 = load_and_scale('Wasilla_bin3.csv', 2)
dfnik1 = load_and_scale('Nikiski_bin1.csv', 386)
dfnik2 = load_and_scale('Nikiski_bin1.csv', 156)
dfnik3 = load_and_scale('Nikiski_bin1.csv', 12)
dfsold1 = load_and_scale('Soldotna_bin1.csv', 4653)
dfsold2 = load_and_scale('Soldotna_bin2.csv', 437)
dfsold3 = load_and_scale('Soldotna_bin3.csv', 9)
dfpalm1 = load_and_scale('Palmer_bin1.csv', 9656)
dfpalm2 = load_and_scale('Palmer_bin2.csv', 115)
dfpalm3 = load_and_scale('Palmer_bin3.csv', 7)
dfhoust1 = load_and_scale('Houston_bin1.csv', 503)
dfhoust2 = load_and_scale('Houston_bin2.csv', 87)
dfhoust3 = load_and_scale('Houston_bin3.csv', 7)
dfanc_bin1 = load_and_scale('ANC_bin1.csv', 14134)
dfanc_bin2 = load_and_scale('ANC_bin2.csv', 472)
dfanc_bin3 = load_and_scale('ANC_bin3.csv', 48)
dfden_bin1 = load_and_scale('Denali_bin1.csv', 2401)
dfden_bin2 = load_and_scale('Denali_bin2.csv', 2)

# Function to merge dataframes without averaging
def merge_dataframes(dfs):
    df_merged = pd.merge(dfs[0][['timestamp', 'HP_kWh']], dfs[1][['timestamp', 'HP_kWh']], on='timestamp', suffixes=('_1', '_2'))
    for i, df in enumerate(dfs[2:], start=3):  # start=3 because we already have _1 and _2
        df_merged = pd.merge(df_merged, df[['timestamp', 'HP_kWh']], on='timestamp')
        df_merged.rename(columns={'HP_kWh': f'HP_kWh_{i}'}, inplace=True)  # Correctly rename the last merged column
    return df_merged

# Merge for each set
dfsal = merge_dataframes([dfsal1, dfsal2, dfsal3])
dffbk = merge_dataframes([dffbk1, dffbk2, dffbk3])
dfnp = merge_dataframes([dfnp1, dfnp2, dfnp3])
dfken = merge_dataframes([dfken1, dfken2, dfken3])
dfwas = merge_dataframes([dfwas1, dfwas2, dfwas3])
dfpalm = merge_dataframes([dfpalm1, dfpalm2, dfpalm3])
dfsold = merge_dataframes([dfsold1, dfsold2, dfsold3])
dfhoust = merge_dataframes([dfhoust1, dfhoust2, dfhoust3])
dfnik = merge_dataframes([dfnik1, dfnik2, dfnik3])
dfanc = merge_dataframes([dfanc_bin1, dfanc_bin2, dfanc_bin3])
dfden = merge_dataframes([dfden_bin1, dfden_bin2])

# Combining dfb1
dfb1 = pd.DataFrame()
dfb1['timestamp'] = dffnsb['timestamp']
dfb1['Total_HP_kWh'] = (
    dffnsb['HP_kWh'] +
    dfkpb['HP_kWh'] +
    dfmatsu['HP_kWh'] +
    dfanc_b1['HP_kWh'] +
    dfden_b1['HP_kWh']
)

# Combining averages for dfc3
dfc3 = pd.DataFrame()
dfc3['timestamp'] = dfsal['timestamp']
dfc3['Total_HP_kWh'] = (
    dfsal.filter(like='HP_kWh').sum(axis=1) +
    dffbk.filter(like='HP_kWh').sum(axis=1) +
    dfnp.filter(like='HP_kWh').sum(axis=1) +
    dfken.filter(like='HP_kWh').sum(axis=1) +
    dfwas.filter(like='HP_kWh').sum(axis=1) +
    dfpalm.filter(like='HP_kWh').sum(axis=1) +
    dfnik.filter(like='HP_kWh').sum(axis=1) +
    dfhoust.filter(like='HP_kWh').sum(axis=1) +
    dfsold.filter(like='HP_kWh').sum(axis=1) +
    dfanc.filter(like='HP_kWh').sum(axis=1) + 
    dfden.filter(like='HP_kWh').sum(axis=1)
)

# Calculate total HP_kWh for each DataFrame
total_HP_kWh_b1 = dfb1['Total_HP_kWh'].sum()
total_HP_kWh_c3 = dfc3['Total_HP_kWh'].sum()

# Calculate the absolute variance and percent difference
variance = abs(total_HP_kWh_b1 - total_HP_kWh_c3)
percent_difference = (variance / ((total_HP_kWh_b1 + total_HP_kWh_c3) / 2)) * 100

# Display the results
print(f"Total HP_kWh in dfb1: {total_HP_kWh_b1}")
print(f"Total HP_kWh in dfc3: {total_HP_kWh_c3}")
print(f"Variance: {variance}")
print(f"Percent Difference: {percent_difference:.2f}%")


Total HP_kWh in dfb1: 33702264189.74153
Total HP_kWh in dfc3: 13614334117.365341
Variance: 20087930072.37619
Percent Difference: 84.91%
